# **단어주머니 벡터의 한계**

> 평균 임베딩의 가장 큰 문제는 **순서 개념**이 없다는 점!
- Yankees defeat Red Sox. (양키스가 레드 삭스에 승리를 거두다.)
- Red Sox defeat Yankees. (레드 삭스가 양키스에 승리를 거두다.)

반대의 정보를 전달하는 위 두 문장을 평균화를 이용해 문장 벡터를 생성하면 동일한 벡터가 나올 것.
심지어 "Sox Red Yankees defeat."도 동일한 문장 임베딩을 얻는다.

> 구절이나 문장의 임베딩을 형성하기 위해 단어 임베딩을 모두 합하거나 평균화하는 접근법을 **단어주머니(bag of words) 접근법**이라 한다. 이 방법은 주머니 안에 단어를 막 던져 넣는 것과 비슷하기 때문에 **순서가 유지되지 않는다**.

> 이번 장의 주제는 **순서를 중요하게 여기는 방식으로 문장 벡터를 생성하는 것**! 그런 문장 벡터를 만들려면 순서가 벡터를 바꾸는 방법이 학습되어야 한다. 이렇게 만들면 언어를 처리하는 방식에서 신경망의 순서 표현이 가능하고, 언어의 순서에 대한 본질적 의미를 파악할 수 있다.

> 시퀀스(ex)문장)에 대한 벡터를 생성하는 가장 유명하고 성공적인 방법 중 하나가 **순환 신경망(RNN:recurrent neural network)**이다. RNN이 동작하는지 보여주기 위해 **단위행렬**로 평균 단어 임베딩을 수행하는 방법부터 보자.

- 단위행렬은 어떤 벡터와 벡터-행렬 곱셈을 수행해도 원본 벡터를 반환함.





In [3]:
import numpy as np

a = np.eye(2)
print(a)
print("-----------------")

b = np.eye(3)
print(b)
print("-----------------")

c = np.eye(4)
print(c)

[[1. 0.]
 [0. 1.]]
-----------------
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
-----------------
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


# **단위행렬을 이용해서 단어 임베딩 총합하기**

- 다른 접근법을 이용해서 같은 논리를 구현해봅시다.

>벡터 하나를 취해서 똑같은 벡터를 출력하는 행렬인 단위행렬을 쓰는 이유가 뭘까?이 예제에서 우리는 더 복잡한 단어 임베딩 총합 방식을 구성해서 신경망이 최종 문장 임베딩을 생성할 때 순서를 계산에 포함하도록 하는 방법을 보여주는 재료로 단위행렬을 사용할 것! 



# **정말 아무것도 바꾸지 않는 행렬**
- 파이썬과 단위행렬을 이용해서 문장 임베딩을 만들어 봅시다.
> 파이썬으로 단위행렬를 다루는 법을 보여줌. 앞 절에서 다뤘던 새로운 문장 벡터 기법을 구현할 것.(이 기법이 동일한 문장 임베딩을 생성함을 증명할 예정)

In [4]:
import numpy as np

a = np.array([1,2,3])
b = np.array([0.1, 0.2, 0.3])
c = np.array([-1, -0.5, 0])
d = np.array([0, 0, 0])

identity = np.eye(3)
print(identity)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [6]:
print(a.dot(identity))
print(b.dot(identity))
print(c.dot(identity))
print(d.dot(identity))

[1. 2. 3.]
[0.1 0.2 0.3]
[-1.  -0.5  0. ]
[0. 0. 0.]


In [7]:
this = np.array([2,4,6])
movie = np.array([10,10,10])
rocks = np.array([1,1,1])

print(this + movie + rocks)
print(this.dot(identity)+movie.dot(identity)+rocks)

[13 15 17]
[13. 15. 17.]


> 앞서 본 그림과 같이 두 방법 모두 **동일한 벡터를 나타낸다**는 것을 보여준다. 단위행렬은 벡터-행렬 곱을 계산했을 때 동일한 벡터를 반환할 것을 보장하는 **유일한 행렬**이다.

# **전이 행렬**
- 손실을 최소화하도록 단위행렬을 바꾸면 어떨까요?

> 우리의 목표 : 문장의 의미에 따라 군집을 이루는 문장 임베딩을 생성하고, 벡터를 이용해서 주어진 문장과 비슷한 의미를 갖는 문장을 찾는 것(특히, **순서**!)

> 앞서 예시를 들었던 
- Red Sox defeat Yankees
- Yankees defeat Red Sox 가 동일한 벡터를 가지게 만들었는데 이 두 문장을 **의미에 관해서는 군집을 이루지만 다른 임베딩**을 만들고 싶음. 이론적으로는 단위행렬을 이용했던 방식으로 문장 임베딩을 생성하면 동일한 단어로 이루어져 있지만 다른 순서로 이루어진 문장은 서로 다른 문장 임베딩을 갖게 됨.

> 단위행렬 대신 어떤 행렬을 사용해야 우리의 목표를 이룰 수 있을까?
- " 신경망 안에서 다른 모든 행렬을 학습하는 것처럼 이 행렬 역시 학습할 것"
무언가 알아내기 위해 만들어진 신경망을 학습시킬 때는 늘 작업이 필요한데, 이 작업은 유용한 단어 벡터와 단위행렬에 대한 수정을 학습함으로써 문장 임베딩을 생성할 수 있도록 해줘야 한다.
- ##  **아는 것   --> 지도 학습 --> 알고싶은 것**
앞서 했던 공란 채우기에 유용한 단어 임베딩을 생성할 때와 비슷하다. 그렇다면 신경망을 학습시켜 입력받은 단어 목록 다음에 따라올 단어를 예측하도록 하면 어떨까?
- ##  **["This", "movie", "was" ] --> 신경망 --> ["great"]**
 

# **유용한 문장 벡터 생성하는 법 학습하기**
- 문장 벡터를 생성하고, 예측하고, 그 일부를 이용해서 문장 벡터를 수정하세요.
> 단어 예측을 할 땐, 현재까지 진행된 문장의 앞부분을 이용해서 문장 임베딩을 형성합니다. (+ 그림처럼)

# **파이썬으로 순전파 하기**
- 이 개념을 이용해서 간단한 순전파를 수행하는 방법을 관찰해봅시다.
> 위 아이디어로 파이썬을 이용한 시험 버전을 만들어봅시다.

In [0]:
import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

word_vects = {}
word_vects['yankees'] = np.array([[0.,0.,0.]]) #----단어 임베딩-----
word_vects['bears'] = np.array([[0.,0.,0.]])
word_vects['braves'] = np.array([[0.,0.,0.]])
word_vects['red'] = np.array([[0.,0.,0.]])
word_vects['socks'] = np.array([[0.,0.,0.]])
word_vects['lose'] = np.array([[0.,0.,0.]])
word_vects['defeat'] = np.array([[0.,0.,0.]])
word_vects['beat'] = np.array([[0.,0.,0.]])
word_vects['tie'] = np.array([[0.,0.,0.]])     #--------------------
sent2output = np.random.rand(3,len(word_vects)) #분류 가중치를 출력하기 위한 문장 임베딩

identity = np.eye(3)  #전이 가중치

In [9]:
layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['socks']   #문장 임베딩 생성
layer_2 = layer_1.dot(identity) + word_vects['defeat']

pred = softmax(layer_2.dot(sent2output))     #모든 어휘에 걸쳐 예측 수행
print(pred)

[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


# **어떻게 여기에 역전파를 넣을까요?**
- 일견 까다로워 보이지만,이미 배운 내용과 다르지 않습니다.

In [0]:
y = np.array([1,0,0,0,0,0,0,0,0]) # target one-hot vector for "yankees"

pred_delta = pred - y
layer_2_delta = pred_delta.dot(sent2output.T)
defeat_delta = layer_2_delta * 1 # can ignore the "1" like prev. chapter
layer_1_delta = layer_2_delta.dot(identity.T)
socks_delta = layer_1_delta * 1 # again... can ignore the "1"
layer_0_delta = layer_1_delta.dot(identity.T)
alpha = 0.01
word_vects['red'] -= layer_0_delta * alpha
word_vects['socks'] -= socks_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha
identity -= np.outer(layer_0,layer_1_delta) * alpha
identity -= np.outer(layer_1,layer_2_delta) * alpha
sent2output -= np.outer(layer_2,pred_delta) * alpha

# **학습시켜 봅시다!**
- 모든 도구를 갖췄으니 장난감 말뭉치로 신경망을 학습시켜봅시다.
